In [170]:
import pandas as pd
import numpy as np


In [171]:
df1 = pd.read_csv("../Data/eth_active_addresses_hr.csv")
df2 = pd.read_csv("../Data/eth_ohlcv_fees_tx_size_fail_rate_hr.csv")
df3 = pd.read_csv("../Data/eth_staking_inflow_hr.csv")
df4 = pd.read_csv("../Data/eth_exchange_deposite_withdraw_tx_hr.csv")
df5 = pd.read_csv("../Data/eth_exchange_netflow_usd_hr.csv")
df6 = pd.read_csv("../Data/eth_RV_hr.csv")

In [172]:
for df in [df1, df2, df3, df4, df5, df6]:
    df['datetime'] = pd.to_datetime(df['datetime'])
df.head()

,datetime,realised_variance,realised_volatility
0,2025-03-20 06:00:00+00:00,0.000026,0.005137
1,2025-03-21 11:00:00+00:00,0.000008,0.002903
2,2025-03-23 23:00:00+00:00,0.000028,0.005261
3,2025-03-25 20:00:00+00:00,0.000026,0.005078
4,2025-03-26 03:00:00+00:00,0.000016,0.003955


In [173]:
# Merge all on datetime
merged = (
    df2
    .merge(df1, on="datetime", how="left")
    .merge(df3, on="datetime", how="left")
    .merge(df4, on="datetime", how="left")
    .merge(df5, on="datetime", how="left")
    .merge(df6, on="datetime", how="left")
    .sort_values("datetime")
)

merged = merged[merged["datetime"] != "2025-10-13 00:00:00+00:00"]
merged['staking_inflow'] = merged['staking_inflow'].fillna(0)
merged.drop(columns=['realised_variance'], inplace=True)

print(merged.shape)
merged.head()

(8208, 18)


,datetime,onchain_volume_usd,transaction_count,avg_gas_fee_usd,avg_priority_fee_usd,fail_rate_percent,open,low,high,close,hourly_return,active_sending_addresses,active_receiving_addresses,staking_inflow,exchange_depositing_count,exchange_withdrawing_count,exchange_netflow_usd,realised_volatility
0,2024-11-05 00:00:00+00:00,9.682857e+07,44719,1.817381,0.738825,2.4,2398.91,2398.91,2409.56,2405.13,NaN,20914.0,12008.0,384.0,1883.0,6340.0,4.572811e+06,0.004405
1,2024-11-05 01:00:00+00:00,9.693977e+07,44513,1.823433,0.819646,2.3,2402.45,2384.78,2410.94,2408.52,0.001408,20028.0,12945.0,63.0,1755.0,5877.0,-9.670579e+05,0.009336
2,2024-11-05 02:00:00+00:00,1.304534e+08,45224,1.783517,0.724244,2.0,2411.90,2402.15,2413.11,2402.73,-0.002407,21410.0,12831.0,10529.0,2085.0,5846.0,1.029868e+07,0.003825
3,2024-11-05 03:00:00+00:00,9.107739e+07,42689,1.623861,0.716063,1.4,2405.17,2403.79,2417.69,2417.44,0.006104,20210.0,12622.0,286.0,1599.0,5429.0,4.960992e+06,0.003505
4,2024-11-05 04:00:00+00:00,9.376725e+07,43063,1.346194,0.550501,1.4,2418.50,2418.50,2431.30,2428.68,0.004639,21076.0,10843.0,65.0,1575.0,5454.0,5.484763e+06,0.003399


In [174]:
# Add moving averages columns
merged['RV_MA_1hr'] = merged['realised_volatility'].shift(1)
merged['RV_MA_3hr'] = merged['RV_MA_1hr'].rolling(window=3).mean()
merged['RV_MA_12hr'] = merged['RV_MA_1hr'].rolling(window=12).mean()
merged["vol_future"] = merged["realised_volatility"].shift(-1)
merged.head()


,datetime,onchain_volume_usd,transaction_count,avg_gas_fee_usd,avg_priority_fee_usd,fail_rate_percent,open,low,high,close,...,active_receiving_addresses,staking_inflow,exchange_depositing_count,exchange_withdrawing_count,exchange_netflow_usd,realised_volatility,RV_MA_1hr,RV_MA_3hr,RV_MA_12hr,vol_future
0,2024-11-05 00:00:00+00:00,9.682857e+07,44719,1.817381,0.738825,2.4,2398.91,2398.91,2409.56,2405.13,...,12008.0,384.0,1883.0,6340.0,4.572811e+06,0.004405,NaN,NaN,NaN,0.009336
1,2024-11-05 01:00:00+00:00,9.693977e+07,44513,1.823433,0.819646,2.3,2402.45,2384.78,2410.94,2408.52,...,12945.0,63.0,1755.0,5877.0,-9.670579e+05,0.009336,0.004405,NaN,NaN,0.003825
2,2024-11-05 02:00:00+00:00,1.304534e+08,45224,1.783517,0.724244,2.0,2411.90,2402.15,2413.11,2402.73,...,12831.0,10529.0,2085.0,5846.0,1.029868e+07,0.003825,0.009336,NaN,NaN,0.003505
3,2024-11-05 03:00:00+00:00,9.107739e+07,42689,1.623861,0.716063,1.4,2405.17,2403.79,2417.69,2417.44,...,12622.0,286.0,1599.0,5429.0,4.960992e+06,0.003505,0.003825,0.005855,NaN,0.003399
4,2024-11-05 04:00:00+00:00,9.376725e+07,43063,1.346194,0.550501,1.4,2418.50,2418.50,2431.30,2428.68,...,10843.0,65.0,1575.0,5454.0,5.484763e+06,0.003399,0.003505,0.005555,NaN,0.003244


In [175]:
merged.columns.tolist()

['datetime',
 'onchain_volume_usd',
 'transaction_count',
 'avg_gas_fee_usd',
 'avg_priority_fee_usd',
 'fail_rate_percent',
 'open',
 'low',
 'high',
 'close',
 'hourly_return',
 'active_sending_addresses',
 'active_receiving_addresses',
 'staking_inflow',
 'exchange_depositing_count',
 'exchange_withdrawing_count',
 'exchange_netflow_usd',
 'realised_volatility',
 'RV_MA_1hr',
 'RV_MA_3hr',
 'RV_MA_12hr',
 'vol_future']

### Log transformation

In [176]:
from scipy.stats import skew
merged_log = merged.copy()
# Select numeric columns
numeric_cols = merged_log.select_dtypes(include=[np.number]).columns

# Compute skewness
skew_vals = merged_log[numeric_cols].apply(lambda x: skew(x.dropna()))
skew_df = pd.DataFrame({"feature": skew_vals.index, "skew": skew_vals.values})
skew_df = skew_df.sort_values(by="skew", ascending=False)

print(skew_df)


                       feature       skew
12              staking_inflow  22.338105
3         avg_priority_fee_usd  20.877469
2              avg_gas_fee_usd  12.340594
17                   RV_MA_1hr   6.919486
16         realised_volatility   6.918769
20                  vol_future   6.918402
18                   RV_MA_3hr   4.654184
13   exchange_depositing_count   4.440219
4            fail_rate_percent   3.887409
0           onchain_volume_usd   2.935166
19                  RV_MA_12hr   2.495189
15        exchange_netflow_usd   1.041357
10    active_sending_addresses   0.789280
14  exchange_withdrawing_count   0.770224
11  active_receiving_addresses   0.604319
1            transaction_count   0.477345
6                          low   0.118388
5                         open   0.115386
8                        close   0.114921
7                         high   0.112566
9                hourly_return  -0.235443


In [177]:
# log transform certain features
log_features = [
    "onchain_volume_usd",
    "avg_gas_fee_usd",
    "avg_priority_fee_usd",
    "staking_inflow",
    "exchange_depositing_count"
]

for col in log_features:
    merged_log[f'{col}_log'] = np.log1p(merged_log[col])
    merged_log = merged_log.drop(columns=[col])

# signed log
merged_log[f'{"exchange_netflow_usd"}_log'] = np.sign(merged_log["exchange_netflow_usd"]) * np.log1p(np.abs(merged_log["exchange_netflow_usd"]))
merged_log = merged_log.drop(columns=["exchange_netflow_usd"])


In [178]:
# Select numeric columns
numeric_cols = merged_log.select_dtypes(include=[np.number]).columns

# Compute skewness
skew_vals = merged_log[numeric_cols].apply(lambda x: skew(x.dropna()))
skew_df = pd.DataFrame({"feature": skew_vals.index, "skew": skew_vals.values})
skew_df = skew_df.sort_values(by="skew", ascending=False)

print(skew_df)

                          feature      skew
11                      RV_MA_1hr  6.919486
10            realised_volatility  6.918769
14                     vol_future  6.918402
12                      RV_MA_3hr  4.654184
1               fail_rate_percent  3.887409
13                     RV_MA_12hr  2.495189
17       avg_priority_fee_usd_log  1.995571
16            avg_gas_fee_usd_log  1.362627
7        active_sending_addresses  0.789280
19  exchange_depositing_count_log  0.772320
9      exchange_withdrawing_count  0.770224
8      active_receiving_addresses  0.604319
0               transaction_count  0.477345
3                             low  0.118388
2                            open  0.115386
5                           close  0.114921
4                            high  0.112566
15         onchain_volume_usd_log  0.057215
6                   hourly_return -0.235443
20       exchange_netflow_usd_log -0.341705
18             staking_inflow_log -0.787987


In [179]:
merged_log.to_csv("../Data/dune_eth_hour.csv", index=False)